In [7]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


In [8]:
# source text
data = """ Piford Technologies is a leading Software Development Company\n
Piford Technologies provide trainings to working professionals and students\n
We are product based and service based company\n
we have one of our office in IT Park, Mohali\n """

In [9]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data


vocab_size = len(tokenizer.word_index) + 1  

print('Vocabulary Size: %d' % vocab_size)



Vocabulary Size: 30


In [11]:
sequences = list()
for i in range(1, len(encoded_data)):
	sequence = encoded_data[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 34


In [12]:
sequences

[[1, 2],
 [2, 7],
 [7, 8],
 [8, 9],
 [9, 10],
 [10, 11],
 [11, 3],
 [3, 1],
 [1, 2],
 [2, 12],
 [12, 13],
 [13, 14],
 [14, 15],
 [15, 16],
 [16, 4],
 [4, 17],
 [17, 5],
 [5, 18],
 [18, 19],
 [19, 6],
 [6, 4],
 [4, 20],
 [20, 6],
 [6, 3],
 [3, 5],
 [5, 21],
 [21, 22],
 [22, 23],
 [23, 24],
 [24, 25],
 [25, 26],
 [26, 27],
 [27, 28],
 [28, 29]]

In [13]:
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

X[:5]



array([1, 2, 7, 8, 9])

In [14]:
y[:5]

array([ 2,  7,  8,  9, 10])

In [15]:
y = to_categorical(y, num_classes=vocab_size)

y[:5]


array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             300       
                                                                 
 lstm (LSTM)                 (None, 50)                12200     
                                                                 
 dense (Dense)               (None, 30)                1530      
                                                                 
Total params: 14,030
Trainable params: 14,030
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [18]:
model.fit(X, y, epochs=10)


Epoch 1/10
2/2 [==============================] - 3s 8ms/step - loss: 3.4007 - accuracy: 0.0588
Epoch 2/10
2/2 [==============================] - 0s 9ms/step - loss: 3.3998 - accuracy: 0.1176
Epoch 3/10
2/2 [==============================] - 0s 9ms/step - loss: 3.3990 - accuracy: 0.1176
Epoch 4/10
2/2 [==============================] - 0s 9ms/step - loss: 3.3982 - accuracy: 0.0588
Epoch 5/10
2/2 [==============================] - 0s 9ms/step - loss: 3.3976 - accuracy: 0.1176
Epoch 6/10
2/2 [==============================] - 0s 13ms/step - loss: 3.3969 - accuracy: 0.0882
Epoch 7/10
2/2 [==============================] - 0s 8ms/step - loss: 3.3963 - accuracy: 0.0882
Epoch 8/10
2/2 [==============================] - 0s 13ms/step - loss: 3.3957 - accuracy: 0.0882
Epoch 9/10
2/2 [==============================] - 0s 8ms/step - loss: 3.3950 - accuracy: 0.0588
Epoch 10/10
2/2 [==============================] - 0s 9ms/step - loss: 3.3945 - accuracy: 0.0588


In [22]:
# enter_text=provide, n_pred=3
# in_text and result =provide

# generate a sequence from the model
def generate_seq(model, tokenizer, enter_text, n_pred):  
	in_text, result = enter_text, enter_text  # 
	# generate a fixed number of words
	for _ in range(n_pred):
        
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = array(encoded)
        
		# predict a word in the vocabulary
		yhat = model.predict(encoded)
        
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

# evaluate
print(generate_seq(model, tokenizer, 'Piford', 6))



ValueError: ignored

In [20]:
print(generate_seq(model, tokenizer, 'service', 3))

NameError: ignored